In [131]:
import numpy as np
import torch
import torch.autograd.functional as F

In [139]:
# camera Rotation Matrix
R = np.array([[1,0,0],
              [0,2,0],
              [0,0,3]])
# camera Pose 4x4
# T_cw = np.array([[1,0,0,1],
#                  [0,1,0,1],
#                  [0,0,1,1],
#                  [0,0,0,1]], dtype=np.float64)

T_cw = np.array([[ 0.8047, -0.3106,  0.5059,  1.0000],
        [ 0.5059,  0.8047, -0.3106,  1.0000],
        [-0.3106,  0.5059,  0.8047,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=np.float64)

# homogenous mu_w
mu_w = np.array([2,3,4,1], dtype=np.float64)

cov_3D = np.array([[1,2,3],
                   [2,4,5],
                   [3,5,9]], dtype=np.float64)

In [140]:
np.matmul(T_cw,mu_w)

array([3.7012, 3.1835, 5.1153, 1.    ])

In [141]:
def pi_func(v):
    '''
    projection function
    v is in homogenous coords
    '''
    # I34 = np.eye(3,4)
    # v_proj=np.divide(v,v[2])
    pi = (np.matmul(np.eye(3,4), v))/v[2]
    # print(v_proj)
    return pi 

def Jacobian_pi_func(v):

    a = np.array([[1, 0, -v[0]/v[2]],
                  [0, 1, -v[1]/v[2]],
                  [0, 0,  0]])
    a = a/v[2]
    return a

def hat_operator(v):
	'''
	Hat operator
	'''
	return np.array([[0    ,-v[2],  v[1]],
                     [v[2] ,    0, -v[0]],
                     [-v[1], v[0],    0]])

In [142]:
def Get_dcovI_dJ(R,cov_3D,mu_c):

    x = mu_c[0]
    y = mu_c[1]
    z = mu_c[2]
    A , B , C , D ,E ,F = cov_3D[0,0], cov_3D[0,1], cov_3D[0,2], cov_3D[1,1], cov_3D[1,2], cov_3D[2,2]
    R_00 ,R_01 ,R_02 ,R_10 ,R_11 ,R_12 ,R_20 ,R_21 ,R_22 = R[0,0], R[0,1], R[0,2], R[1,0], R[1,1], R[1,2], R[2,0], R[2,1], R[2,2]
    
    
    temp = np.array([[R_00*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_01*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_02*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)) - x*(R_20*(A*R_00 + B*R_01 + C*R_02) + R_21*(B*R_00 + D*R_01 + E*R_02) + R_22*(C*R_00 + E*R_01 + F*R_02))/z**2 + (R_00*(A*R_00 + B*R_01 + C*R_02) + R_01*(B*R_00 + D*R_01 + E*R_02) + R_02*(C*R_00 + E*R_01 + F*R_02))/z, R_10*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_11*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_12*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)) - x*(R_20*(A*R_10 + B*R_11 + C*R_12) + R_21*(B*R_10 + D*R_11 + E*R_12) + R_22*(C*R_10 + E*R_11 + F*R_12))/z**2 + (R_00*(A*R_10 + B*R_11 + C*R_12) + R_01*(B*R_10 + D*R_11 + E*R_12) + R_02*(C*R_10 + E*R_11 + F*R_12))/z, R_20*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_21*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_22*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)) - x*(R_20*(A*R_20 + B*R_21 + C*R_22) + R_21*(B*R_20 + D*R_21 + E*R_22) + R_22*(C*R_20 + E*R_21 + F*R_22))/z**2 + (R_00*(A*R_20 + B*R_21 + C*R_22) + R_01*(B*R_20 + D*R_21 + E*R_22) + R_02*(C*R_20 + E*R_21 + F*R_22))/z, 0, 0, 0], 
                     [-y*(R_20*(A*R_00 + B*R_01 + C*R_02) + R_21*(B*R_00 + D*R_01 + E*R_02) + R_22*(C*R_00 + E*R_01 + F*R_02))/z**2 + (R_10*(A*R_00 + B*R_01 + C*R_02) + R_11*(B*R_00 + D*R_01 + E*R_02) + R_12*(C*R_00 + E*R_01 + F*R_02))/z, -y*(R_20*(A*R_10 + B*R_11 + C*R_12) + R_21*(B*R_10 + D*R_11 + E*R_12) + R_22*(C*R_10 + E*R_11 + F*R_12))/z**2 + (R_10*(A*R_10 + B*R_11 + C*R_12) + R_11*(B*R_10 + D*R_11 + E*R_12) + R_12*(C*R_10 + E*R_11 + F*R_12))/z, -y*(R_20*(A*R_20 + B*R_21 + C*R_22) + R_21*(B*R_20 + D*R_21 + E*R_22) + R_22*(C*R_20 + E*R_21 + F*R_22))/z**2 + (R_10*(A*R_20 + B*R_21 + C*R_22) + R_11*(B*R_20 + D*R_21 + E*R_22) + R_12*(C*R_20 + E*R_21 + F*R_22))/z, R_00*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_01*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_02*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)), R_10*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_11*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_12*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)), R_20*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_21*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_22*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2))], 
                     [R_00*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_01*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_02*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)), R_10*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_11*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_12*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)), R_20*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_21*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_22*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)), -x*(R_20*(A*R_00 + B*R_01 + C*R_02) + R_21*(B*R_00 + D*R_01 + E*R_02) + R_22*(C*R_00 + E*R_01 + F*R_02))/z**2 + (R_00*(A*R_00 + B*R_01 + C*R_02) + R_01*(B*R_00 + D*R_01 + E*R_02) + R_02*(C*R_00 + E*R_01 + F*R_02))/z, -x*(R_20*(A*R_10 + B*R_11 + C*R_12) + R_21*(B*R_10 + D*R_11 + E*R_12) + R_22*(C*R_10 + E*R_11 + F*R_12))/z**2 + (R_00*(A*R_10 + B*R_11 + C*R_12) + R_01*(B*R_10 + D*R_11 + E*R_12) + R_02*(C*R_10 + E*R_11 + F*R_12))/z, -x*(R_20*(A*R_20 + B*R_21 + C*R_22) + R_21*(B*R_20 + D*R_21 + E*R_22) + R_22*(C*R_20 + E*R_21 + F*R_22))/z**2 + (R_00*(A*R_20 + B*R_21 + C*R_22) + R_01*(B*R_20 + D*R_21 + E*R_22) + R_02*(C*R_20 + E*R_21 + F*R_22))/z], 
                     [0, 0, 0, R_00*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_01*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_02*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)) - y*(R_20*(A*R_00 + B*R_01 + C*R_02) + R_21*(B*R_00 + D*R_01 + E*R_02) + R_22*(C*R_00 + E*R_01 + F*R_02))/z**2 + (R_10*(A*R_00 + B*R_01 + C*R_02) + R_11*(B*R_00 + D*R_01 + E*R_02) + R_12*(C*R_00 + E*R_01 + F*R_02))/z, R_10*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_11*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_12*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)) - y*(R_20*(A*R_10 + B*R_11 + C*R_12) + R_21*(B*R_10 + D*R_11 + E*R_12) + R_22*(C*R_10 + E*R_11 + F*R_12))/z**2 + (R_10*(A*R_10 + B*R_11 + C*R_12) + R_11*(B*R_10 + D*R_11 + E*R_12) + R_12*(C*R_10 + E*R_11 + F*R_12))/z, R_20*(A*(R_10/z - R_20*y/z**2) + B*(R_11/z - R_21*y/z**2) + C*(R_12/z - R_22*y/z**2)) + R_21*(B*(R_10/z - R_20*y/z**2) + D*(R_11/z - R_21*y/z**2) + E*(R_12/z - R_22*y/z**2)) + R_22*(C*(R_10/z - R_20*y/z**2) + E*(R_11/z - R_21*y/z**2) + F*(R_12/z - R_22*y/z**2)) - y*(R_20*(A*R_20 + B*R_21 + C*R_22) + R_21*(B*R_20 + D*R_21 + E*R_22) + R_22*(C*R_20 + E*R_21 + F*R_22))/z**2 + (R_10*(A*R_20 + B*R_21 + C*R_22) + R_11*(B*R_20 + D*R_21 + E*R_22) + R_12*(C*R_20 + E*R_21 + F*R_22))/z]])
    return temp

In [136]:
Get_dcovI_dJ(R, cov_3D, mu_c)

array([[ -1.76,  -5.6 , -15.84,   0.  ,   0.  ,   0.  ],
       [ -0.64,  -1.6 ,  -6.96,  -0.88,  -2.8 ,  -7.92],
       [ -0.64,  -1.6 ,  -6.96,  -0.88,  -2.8 ,  -7.92],
       [  0.  ,   0.  ,   0.  ,  -1.28,  -3.2 , -13.92]])

In [143]:
def GetAnalyticalJcobian(T_cw, mu_w):
    '''
    T_cw : Camera pose in world frame
    mu_w : Mean of 3D gaussian
    '''
    R = T_cw[0:3,0:3]
    
    mu_c = np.matmul(T_cw,mu_w)
    print("mu_c:\n", mu_c)
    
    a = 1/mu_c[2]
    b = -mu_c[0]/(mu_c[2]**2)
    c = -mu_c[1]/(mu_c[2]**2)
    
    dmuI_dmuC = np.array([[a , 0 , b],
                          [0,  a , c]])
    print("dmuI_dmuC: \n", dmuI_dmuC)
    
    dmuC_dTcw = np.hstack((np.eye(3),-hat_operator(mu_c)))
    print("dmuC_dTcw: \n", dmuC_dTcw)
    
    dmuI_dTcw = np.matmul(dmuI_dmuC,dmuC_dTcw) # Equation 3
    print("dmuI_dTcw: \n", dmuI_dTcw)
    
    # 2nd term of eqution 4 
    
    dW_dTcw = np.zeros([9,6])
    dW_dTcw[0:3,3:6] = -hat_operator(R[:,0])
    dW_dTcw[3:6,3:6] = -hat_operator(R[:,1])
    dW_dTcw[6:9,3:6] = -hat_operator(R[:,2])
    
    print("dW_dTcw: \n",dW_dTcw)
    
    # dmuI_dW
    dcovI_dW = np.array([[a*(cov_3D[0,0]*R[0,0]*a + cov_3D[0,0]*(R[0,0]*a + R[2,0]*b) + cov_3D[0,1]*R[0,1]*a + cov_3D[0,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[0,2]*R[0,2]*a + cov_3D[0,2]*(R[0,2]*a + R[2,2]*b)) + b*(cov_3D[0,0]*R[2,0]*a + cov_3D[0,1]*R[2,1]*a + cov_3D[0,2]*R[2,2]*a), 0, a*(cov_3D[0,0]*R[0,0]*b + cov_3D[0,1]*R[0,1]*b + cov_3D[0,2]*R[0,2]*b) + b*(cov_3D[0,0]*R[2,0]*b + cov_3D[0,0]*(R[0,0]*a + R[2,0]*b) + cov_3D[0,1]*R[2,1]*b + cov_3D[0,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[0,2]*R[2,2]*b + cov_3D[0,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,1]*R[0,0]*a + cov_3D[0,1]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,1]*R[0,1]*a + cov_3D[1,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[1,2]*R[0,2]*a + cov_3D[1,2]*(R[0,2]*a + R[2,2]*b)) + b*(cov_3D[0,1]*R[2,0]*a + cov_3D[1,1]*R[2,1]*a + cov_3D[1,2]*R[2,2]*a), 0, a*(cov_3D[0,1]*R[0,0]*b + cov_3D[1,1]*R[0,1]*b + cov_3D[1,2]*R[0,2]*b) + b*(cov_3D[0,1]*R[2,0]*b + cov_3D[0,1]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,1]*R[2,1]*b + cov_3D[1,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[1,2]*R[2,2]*b + cov_3D[1,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,2]*R[0,0]*a + cov_3D[0,2]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,2]*R[0,1]*a + cov_3D[1,2]*(R[0,1]*a + R[2,1]*b) + cov_3D[2,2]*R[0,2]*a + cov_3D[2,2]*(R[0,2]*a + R[2,2]*b)) + b*(cov_3D[0,2]*R[2,0]*a + cov_3D[1,2]*R[2,1]*a + cov_3D[2,2]*R[2,2]*a), 0, a*(cov_3D[0,2]*R[0,0]*b + cov_3D[1,2]*R[0,1]*b + cov_3D[2,2]*R[0,2]*b) + b*(cov_3D[0,2]*R[2,0]*b + cov_3D[0,2]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,2]*R[2,1]*b + cov_3D[1,2]*(R[0,1]*a + R[2,1]*b) + cov_3D[2,2]*R[2,2]*b + cov_3D[2,2]*(R[0,2]*a + R[2,2]*b))],
                         [a*(cov_3D[0,0]*R[1,0]*a + cov_3D[0,1]*R[1,1]*a + cov_3D[0,2]*R[1,2]*a) + c*(cov_3D[0,0]*R[2,0]*a + cov_3D[0,1]*R[2,1]*a + cov_3D[0,2]*R[2,2]*a), a*(cov_3D[0,0]*(R[0,0]*a + R[2,0]*b) + cov_3D[0,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[0,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,0]*R[1,0]*b + cov_3D[0,1]*R[1,1]*b + cov_3D[0,2]*R[1,2]*b) + c*(cov_3D[0,0]*R[2,0]*b + cov_3D[0,0]*(R[0,0]*a + R[2,0]*b) + cov_3D[0,1]*R[2,1]*b + cov_3D[0,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[0,2]*R[2,2]*b + cov_3D[0,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,1]*R[1,0]*a + cov_3D[1,1]*R[1,1]*a + cov_3D[1,2]*R[1,2]*a) + c*(cov_3D[0,1]*R[2,0]*a + cov_3D[1,1]*R[2,1]*a + cov_3D[1,2]*R[2,2]*a), a*(cov_3D[0,1]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[1,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,1]*R[1,0]*b + cov_3D[1,1]*R[1,1]*b + cov_3D[1,2]*R[1,2]*b) + c*(cov_3D[0,1]*R[2,0]*b + cov_3D[0,1]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,1]*R[2,1]*b + cov_3D[1,1]*(R[0,1]*a + R[2,1]*b) + cov_3D[1,2]*R[2,2]*b + cov_3D[1,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,2]*R[1,0]*a + cov_3D[1,2]*R[1,1]*a + cov_3D[2,2]*R[1,2]*a) + c*(cov_3D[0,2]*R[2,0]*a + cov_3D[1,2]*R[2,1]*a + cov_3D[2,2]*R[2,2]*a), a*(cov_3D[0,2]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,2]*(R[0,1]*a + R[2,1]*b) + cov_3D[2,2]*(R[0,2]*a + R[2,2]*b)), a*(cov_3D[0,2]*R[1,0]*b + cov_3D[1,2]*R[1,1]*b + cov_3D[2,2]*R[1,2]*b) + c*(cov_3D[0,2]*R[2,0]*b + cov_3D[0,2]*(R[0,0]*a + R[2,0]*b) + cov_3D[1,2]*R[2,1]*b + cov_3D[1,2]*(R[0,1]*a + R[2,1]*b) + cov_3D[2,2]*R[2,2]*b + cov_3D[2,2]*(R[0,2]*a + R[2,2]*b))],
                         [a*(cov_3D[0,0]*(R[1,0]*a + R[2,0]*c) + cov_3D[0,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[0,2]*(R[1,2]*a + R[2,2]*c)), a*(cov_3D[0,0]*R[0,0]*a + cov_3D[0,1]*R[0,1]*a + cov_3D[0,2]*R[0,2]*a) + b*(cov_3D[0,0]*R[2,0]*a + cov_3D[0,1]*R[2,1]*a + cov_3D[0,2]*R[2,2]*a), a*(cov_3D[0,0]*R[0,0]*c + cov_3D[0,1]*R[0,1]*c + cov_3D[0,2]*R[0,2]*c) + b*(cov_3D[0,0]*R[2,0]*c + cov_3D[0,0]*(R[1,0]*a + R[2,0]*c) + cov_3D[0,1]*R[2,1]*c + cov_3D[0,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[0,2]*R[2,2]*c + cov_3D[0,2]*(R[1,2]*a + R[2,2]*c)), a*(cov_3D[0,1]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[1,2]*(R[1,2]*a + R[2,2]*c)), a*(cov_3D[0,1]*R[0,0]*a + cov_3D[1,1]*R[0,1]*a + cov_3D[1,2]*R[0,2]*a) + b*(cov_3D[0,1]*R[2,0]*a + cov_3D[1,1]*R[2,1]*a + cov_3D[1,2]*R[2,2]*a), a*(cov_3D[0,1]*R[0,0]*c + cov_3D[1,1]*R[0,1]*c + cov_3D[1,2]*R[0,2]*c) + b*(cov_3D[0,1]*R[2,0]*c + cov_3D[0,1]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,1]*R[2,1]*c + cov_3D[1,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[1,2]*R[2,2]*c + cov_3D[1,2]*(R[1,2]*a + R[2,2]*c)), a*(cov_3D[0,2]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,2]*(R[1,1]*a + R[2,1]*c) + cov_3D[2,2]*(R[1,2]*a + R[2,2]*c)), a*(cov_3D[0,2]*R[0,0]*a + cov_3D[1,2]*R[0,1]*a + cov_3D[2,2]*R[0,2]*a) + b*(cov_3D[0,2]*R[2,0]*a + cov_3D[1,2]*R[2,1]*a + cov_3D[2,2]*R[2,2]*a), a*(cov_3D[0,2]*R[0,0]*c + cov_3D[1,2]*R[0,1]*c + cov_3D[2,2]*R[0,2]*c) + b*(cov_3D[0,2]*R[2,0]*c + cov_3D[0,2]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,2]*R[2,1]*c + cov_3D[1,2]*(R[1,1]*a + R[2,1]*c) + cov_3D[2,2]*R[2,2]*c + cov_3D[2,2]*(R[1,2]*a + R[2,2]*c))],
                         [0, a*(cov_3D[0,0]*R[1,0]*a + cov_3D[0,0]*(R[1,0]*a + R[2,0]*c) + cov_3D[0,1]*R[1,1]*a + cov_3D[0,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[0,2]*R[1,2]*a + cov_3D[0,2]*(R[1,2]*a + R[2,2]*c)) + c*(cov_3D[0,0]*R[2,0]*a + cov_3D[0,1]*R[2,1]*a + cov_3D[0,2]*R[2,2]*a), a*(cov_3D[0,0]*R[1,0]*c + cov_3D[0,1]*R[1,1]*c + cov_3D[0,2]*R[1,2]*c) + c*(cov_3D[0,0]*R[2,0]*c + cov_3D[0,0]*(R[1,0]*a + R[2,0]*c) + cov_3D[0,1]*R[2,1]*c + cov_3D[0,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[0,2]*R[2,2]*c + cov_3D[0,2]*(R[1,2]*a + R[2,2]*c)), 0, a*(cov_3D[0,1]*R[1,0]*a + cov_3D[0,1]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,1]*R[1,1]*a + cov_3D[1,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[1,2]*R[1,2]*a + cov_3D[1,2]*(R[1,2]*a + R[2,2]*c)) + c*(cov_3D[0,1]*R[2,0]*a + cov_3D[1,1]*R[2,1]*a + cov_3D[1,2]*R[2,2]*a), a*(cov_3D[0,1]*R[1,0]*c + cov_3D[1,1]*R[1,1]*c + cov_3D[1,2]*R[1,2]*c) + c*(cov_3D[0,1]*R[2,0]*c + cov_3D[0,1]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,1]*R[2,1]*c + cov_3D[1,1]*(R[1,1]*a + R[2,1]*c) + cov_3D[1,2]*R[2,2]*c + cov_3D[1,2]*(R[1,2]*a + R[2,2]*c)), 0, a*(cov_3D[0,2]*R[1,0]*a + cov_3D[0,2]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,2]*R[1,1]*a + cov_3D[1,2]*(R[1,1]*a + R[2,1]*c) + cov_3D[2,2]*R[1,2]*a + cov_3D[2,2]*(R[1,2]*a + R[2,2]*c)) + c*(cov_3D[0,2]*R[2,0]*a + cov_3D[1,2]*R[2,1]*a + cov_3D[2,2]*R[2,2]*a), a*(cov_3D[0,2]*R[1,0]*c + cov_3D[1,2]*R[1,1]*c + cov_3D[2,2]*R[1,2]*c) + c*(cov_3D[0,2]*R[2,0]*c + cov_3D[0,2]*(R[1,0]*a + R[2,0]*c) + cov_3D[1,2]*R[2,1]*c + cov_3D[1,2]*(R[1,1]*a + R[2,1]*c) + cov_3D[2,2]*R[2,2]*c + cov_3D[2,2]*(R[1,2]*a + R[2,2]*c))]])
    print("dcovI_dW: \n", dcovI_dW)
    
    second_term = np.matmul(dcovI_dW,dW_dTcw)
    # print(second_term)
    
    
    # First term of eqution 4
    # dcovI_dJ = np.array([[R[0,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[0,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[0,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) - mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[2,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[2,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[0,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[0,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2], 0, R[2,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[2,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[2,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) - mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[2,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[2,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[0,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[0,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2], 0, R[0,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[0,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[0,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) - mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[2,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[2,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[0,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[0,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2], 0],
    #   [R[1,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[1,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[1,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) - mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[2,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[2,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[1,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[1,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2], 0, -mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[2,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[2,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[1,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[1,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2], 0, R[1,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[1,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[1,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) - mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[2,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[2,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[0,0] + cov_3D[0,1]*R[0,1] + cov_3D[0,2]*R[0,2]) + R[1,1]*(cov_3D[0,1]*R[0,0] + cov_3D[1,1]*R[0,1] + cov_3D[1,2]*R[0,2]) + R[1,2]*(cov_3D[0,2]*R[0,0] + cov_3D[1,2]*R[0,1] + cov_3D[2,2]*R[0,2]))/mu_c[2], R[2,0]*(cov_3D[0,0]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[0,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[0,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[2,1]*(cov_3D[0,1]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,1]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2)) + R[2,2]*(cov_3D[0,2]*(R[0,0]/mu_c[2] - R[2,0]*mu_c[0]/mu_c[2]**2) + cov_3D[1,2]*(R[0,1]/mu_c[2] - R[2,1]*mu_c[0]/mu_c[2]**2) + cov_3D[2,2]*(R[0,2]/mu_c[2] - R[2,2]*mu_c[0]/mu_c[2]**2))], 
    #   [R[0,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[0,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[0,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) - mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[2,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[2,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[0,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[0,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2], 0, R[2,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[2,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[2,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)), 0, R[0,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[0,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[0,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) - mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[2,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[2,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[0,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[0,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2], -mu_c[0]*(R[2,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[2,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[2,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]**2 + (R[0,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[0,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[0,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]], 
    #   [R[1,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[1,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[1,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) - mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[2,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[2,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[1,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[1,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2], 0, 0, 0, R[1,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[1,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[1,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) - mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[2,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[2,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[1,0] + cov_3D[0,1]*R[1,1] + cov_3D[0,2]*R[1,2]) + R[1,1]*(cov_3D[0,1]*R[1,0] + cov_3D[1,1]*R[1,1] + cov_3D[1,2]*R[1,2]) + R[1,2]*(cov_3D[0,2]*R[1,0] + cov_3D[1,2]*R[1,1] + cov_3D[2,2]*R[1,2]))/mu_c[2], R[2,0]*(cov_3D[0,0]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[0,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[0,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[2,1]*(cov_3D[0,1]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,1]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) + R[2,2]*(cov_3D[0,2]*(R[1,0]/mu_c[2] - R[2,0]*mu_c[1]/mu_c[2]**2) + cov_3D[1,2]*(R[1,1]/mu_c[2] - R[2,1]*mu_c[1]/mu_c[2]**2) + cov_3D[2,2]*(R[1,2]/mu_c[2] - R[2,2]*mu_c[1]/mu_c[2]**2)) - mu_c[1]*(R[2,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[2,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[2,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]**2 + (R[1,0]*(cov_3D[0,0]*R[2,0] + cov_3D[0,1]*R[2,1] + cov_3D[0,2]*R[2,2]) + R[1,1]*(cov_3D[0,1]*R[2,0] + cov_3D[1,1]*R[2,1] + cov_3D[1,2]*R[2,2]) + R[1,2]*(cov_3D[0,2]*R[2,0] + cov_3D[1,2]*R[2,1] + cov_3D[2,2]*R[2,2]))/mu_c[2]]])

    dcovI_dJ = Get_dcovI_dJ(R, cov_3D, mu_c)
    
    print("dcovI_dJ: ", dcovI_dJ)
    
    dJ_dmu_c = np.array([[0, 0, -1/mu_c[2]**2], [0, 0, 0], [-1/mu_c[2]**2, 0, 2*mu_c[0]/mu_c[2]**3], [0, 0, 0], [0, 0, -1/mu_c[2]**2], [0, -1/mu_c[2]**2, 2*mu_c[1]/mu_c[2]**3]])
    print("dJ_dmu_c: \n", dJ_dmu_c)
    
    
    First_term = np.matmul(dcovI_dJ,np.matmul(dJ_dmu_c,dmuC_dTcw))
    
    print('First_term: \n',First_term)
    print('seond_term: \n',second_term)
    
    dcovI_dTcw = First_term+second_term
    # print(dcovI_dTcw)

    return dmuI_dTcw, dcovI_dTcw

In [144]:
dmuI_dTcw, dcovI_dTcw = GetAnalyticalJcobian(T_cw,mu_w)
print("dmuI_dTcw: \n",dmuI_dTcw)
print("dcovI_dTcw: \n" ,dcovI_dTcw)

mu_c:
 [3.7012 3.1835 5.1153 1.    ]
dmuI_dmuC: 
 [[ 0.19549196  0.         -0.14144915]
 [ 0.          0.19549196 -0.12166415]]
dmuC_dTcw: 
 [[ 1.      0.      0.     -0.      5.1153 -3.1835]
 [ 0.      1.      0.     -5.1153 -0.      3.7012]
 [ 0.      0.      1.      3.1835 -3.7012 -0.    ]]
dmuI_dTcw: 
 [[ 0.19549196  0.         -0.14144915 -0.45030336  1.52353159 -0.62234864]
 [ 0.          0.19549196 -0.12166415 -1.38731783  0.45030336  0.72355483]]
dW_dTcw: 
 [[ 0.      0.      0.     -0.     -0.3106 -0.5059]
 [ 0.      0.      0.      0.3106 -0.      0.8047]
 [ 0.      0.      0.      0.5059 -0.8047 -0.    ]
 [ 0.      0.      0.     -0.      0.5059 -0.8047]
 [ 0.      0.      0.     -0.5059 -0.     -0.3106]
 [ 0.      0.      0.      0.8047  0.3106 -0.    ]
 [ 0.      0.      0.     -0.      0.8047  0.3106]
 [ 0.      0.      0.     -0.8047 -0.      0.5059]
 [ 0.      0.      0.     -0.3106 -0.5059 -0.    ]]
dcovI_dW: 
 [[-0.04225974  0.          0.03057724 -0.07868414  0.    

In [75]:
dmuI_dTcw, dcovI_dTcw = GetAnalyticalJcobian(T_cw,mu_w)
print("dmuI_dTcw: \n",dmuI_dTcw)
print("dcovI_dTcw: \n" ,dcovI_dTcw)

mu_c:
 [3. 4. 5. 1.]
dmuI_dmuC: 
 [[ 0.2   0.   -0.12]
 [ 0.    0.2  -0.16]]
dmuC_dTcw: 
 [[ 1.  0.  0. -0.  4. -3.]
 [ 0.  1.  0. -4. -0.  2.]
 [ 0.  0.  1.  3. -2. -0.]]
dmuI_dTcw: 
 [[ 0.2   0.   -0.12 -0.36  1.04 -0.6 ]
 [ 0.    0.2  -0.16 -1.28  0.32  0.4 ]]
dJ_dmu_c: 
 [[ 0.     0.    -0.04 ]
 [ 0.     0.     0.   ]
 [-0.04   0.     0.048]
 [ 0.     0.     0.   ]
 [ 0.     0.    -0.04 ]
 [ 0.    -0.04   0.064]]
First_term: 
 [[ 0.0384   0.      -0.02048 -0.06144  0.19456 -0.1152 ]
 [ 0.0176   0.0192  -0.02944 -0.16512  0.12928 -0.0144 ]
 [ 0.0176   0.0192  -0.02944 -0.16512  0.12928 -0.0144 ]
 [ 0.       0.0352  -0.05632 -0.30976  0.11264  0.0704 ]]
seond_term: 
 [[ 0.      0.      0.      0.048  -0.2304  0.08  ]
 [ 0.      0.      0.      0.128  -0.1232 -0.032 ]
 [ 0.      0.      0.      0.128  -0.1232 -0.032 ]
 [ 0.      0.      0.      0.176  -0.0256 -0.032 ]]
dmuI_dTcw: 
 [[ 0.2   0.   -0.12 -0.36  1.04 -0.6 ]
 [ 0.    0.2  -0.16 -1.28  0.32  0.4 ]]
dcovI_dTcw: 
 [[ 0.0384  

In [53]:
# With Float64 precision

dmuI_dTcw, dcovI_dTcw = GetAnalyticalJcobian(T_cw,mu_w)
print("dmuI_dTcw: \n",dmuI_dTcw)
print("dcovI_dTcw: \n" ,dcovI_dTcw)

dmuI_dTcw: 
 [[ 0.2   0.   -0.12 -0.48  1.36 -0.8 ]
 [ 0.    0.2  -0.12 -1.48  0.36  0.6 ]]
dcovI_dTcw: 
 [[ 0.0384   0.      -0.02048 -0.03392  0.02304 -0.0736 ]
 [ 0.0176   0.0192  -0.02944 -0.08576  0.05312 -0.0448 ]
 [ 0.0176   0.0192  -0.02944 -0.08576  0.05312 -0.0448 ]
 [ 0.       0.0352  -0.05632 -0.22528  0.14336  0.0736 ]]


# Re-verify

In [ ]:
DcovI_DJ = np.array([[R_00*(A*(R_00/z - R_20*x/z**2) + B*(R_01/z - R_21*x/z**2) + C*(R_02/z - R_22*x/z**2)) + R_01*(B*(R_00/z - R_20*x/z**2) + D*(R_01/z - R_21*x/z**2) + E*(R_02/z - R_22*x/z**2)) + R_02*(C*(R_00/z - R_20*x/z**2) + E*(R_01/z - R_21*x/z**2) + F*(R_02/z - R_22*x/z**2)) - x*(R_20*(A*R_00 + B*R_01 + C*R_02) + R_21*(B*R_00 + D*R_01 + E*R_02) + R_22*(C*R_00 + E*R_01 + F*R_02))/z**2 + (R_00*(A*R_00 + B*R_01 + C*R_02) + R_01*(B*R_00 + D*R_01 + E*R_02) + R_02*(C*R_00 + E*R_01 + F*R_02))/z,_,_],[_,_,_]])

# Verify that the jacobian is correct

In [7]:
def pi_func_tensor(v):
    '''
    projection function
    v is in homogenous coords
    '''
    I34 = torch.eye(3,4, requires_grad=True, dtype=torch.float32)
    pi = (torch.matmul(I34, v))/v[2]
    return pi

def hat_operator_tensor(v):
	'''
	Hat operator
	'''
	return torch.tensor([[0,-v[2],v[1]],
                         [v[2],0,-v[0]],
                         [-v[1],v[0],0]], requires_grad=True,  dtype=torch.float32)

def se3_hat(v):
    '''
    Dot operator
    '''
    trans = v[0:3]
    rot = v[3::]
    
    top = torch.cat((hat_operator_tensor(rot),torch.reshape(trans, (3,1)) ), dim=1)
    bottom = torch.cat((torch.zeros(1,3), torch.zeros(1,1)), dim=1)

    big_tensor = torch.cat((top, bottom), dim=0)
    
    return big_tensor

def Jacobian_pi_func_tensor(v):

    # a = torch.tensor([[1, 0, -v[0]/v[2]],
    #               [0, 1, -v[1]/v[2]],
    #               [0, 0,  0]], requires_grad=True,  dtype=torch.float32)
    
    a = torch.tensor([[1, 0, -v[0]/v[2]],
                  [0, 1, -v[1]/v[2]]], requires_grad=True,  dtype=torch.float32)
    
    a = a/v[2]
    return a

#### Convert numpy array to tensors

In [8]:
# camera Pose 4x4
T_cw_tensor = torch.from_numpy(T_cw).float().requires_grad_(True)
# homogenous mu_w
mu_w_tensor = torch.from_numpy(mu_w).float().requires_grad_(True)

cov_3D_tensor = torch.from_numpy(cov_3D).float().requires_grad_(True)

In [9]:
mu_w_tensor

tensor([2., 3., 4., 1.], requires_grad=True)

In [10]:
mu_cam_tensor = torch.matmul(T_cw_tensor,mu_w_tensor)
mu_cam_tensor

tensor([3., 4., 5., 1.], grad_fn=<MvBackward0>)

In [11]:
mu_I_tensor = pi_func_tensor(mu_cam_tensor)

In [12]:
J_tensor = Jacobian_pi_func_tensor(mu_cam_tensor)
J_tensor

tensor([[ 0.2000,  0.0000, -0.1200],
        [ 0.0000,  0.2000, -0.1600]], grad_fn=<DivBackward0>)

In [13]:
W = T_cw_tensor[0:3,0:3]
JW = torch.matmul(J_tensor,W)
print(JW)

cov_I_tensor = torch.matmul(JW,torch.matmul(cov_3D_tensor,(JW.transpose(0,1))))
print(cov_I_tensor)

tensor([[ 0.2000,  0.0000, -0.1200],
        [ 0.0000,  0.2000, -0.1600]], grad_fn=<MmBackward0>)
tensor([[0.0256, 0.0368],
        [0.0368, 0.0704]], grad_fn=<MmBackward0>)


In [14]:
# Project the covariance to the image plane
J = Jacobian_pi_func(np.matmul(T_cw,mu_w))
W = T_cw[0:3,0:3]
JW = J@W
# JW
cov_I = JW@cov_3D@(np.transpose(JW))
cov_I

array([[0.0256, 0.0368, 0.    ],
       [0.0368, 0.0704, 0.    ],
       [0.    , 0.    , 0.    ]])

In [15]:
J

array([[ 0.2 ,  0.  , -0.12],
       [ 0.  ,  0.2 , -0.16],
       [ 0.  ,  0.  ,  0.  ]])

import torch


def func(x):
  return x**2, 2*x

inputs = torch.tensor([3.0], requires_grad=True)
jacobian = F.jacobian(func, inputs)
print(jacobian)
# Expected output: (tensor([4.]), tensor([2.]))

In [16]:
r_vec = torch.tensor([0.0, 0.0, 0.0], requires_grad=True)
t_vec = torch.tensor([1.0, 1.0, 1.0], requires_grad=True)

In [30]:
vec = torch.tensor([1.0, 1.0, 1.0, 0.0, 0.0, 0.0], requires_grad=True)

In [18]:
def func(r_vec,t_vec, mu_w):
    xi = torch.tensor([
        [r_vec[3], -r_vec[5],  r_vec[4],  t_vec[0]],  # ω_x, ω_y, ω_z, v_x
        [r_vec[5],  r_vec[4], -r_vec[3],  t_vec[1]],  # ω_y, ω_z, ω_x, v_y
        [-r_vec[4], r_vec[3],  r_vec[5],  t_vec[2]],  # ω_z, ω_x, ω_y, v_z
        [0.,  0.,  0.,  0.]   # Homogeneous row
    ],requires_grad=True)
    
    T_cw = torch.matrix_exp(xi)
    mu_I = pi_func_tensor(torch.matmul(T_cw,mu_w))
    # print(mu_I)
    return mu_I

In [31]:
def func_2(vec, mu_w):
    # xi = torch.tensor([
    #     [vec[0], -vec[2],  vec[1],  vec[3]],  # ω_x, ω_y, ω_z, v_x
    #     [vec[2],  vec[1], -vec[0],  vec[4]],  # ω_y, ω_z, ω_x, v_y
    #     [-vec[1], vec[0],  vec[2],  vec[5]],  # ω_z, ω_x, ω_y, v_z
    #     [0.,  0.,  0.,  0.]   # Homogeneous row
    # ])

    xi = torch.stack([
    torch.stack([vec[3], -vec[5],  vec[4],  vec[0]]),
    torch.stack([vec[5],  vec[4], -vec[3],  vec[1]]),
    torch.stack([-vec[4], vec[3],  vec[5],  vec[2]]),
    torch.zeros(4)  # Last row
    ])
    xi.requires_grad_(True)
    
    T_cw = torch.matrix_exp(xi)
    mu_cam = torch.matmul(T_cw,mu_w)
    print(mu_cam)

    I34 = torch.eye(3,4, requires_grad=True, dtype=torch.float32)
    mu_I = (torch.matmul(I34, mu_cam))/mu_cam[2]
    print(mu_I)
    
    # print(mu_I)
    return mu_I

In [20]:
func(r_vec, t_vec, mu_w_tensor)

tensor([0.6000, 0.8000, 1.0000], grad_fn=<DivBackward0>)

In [28]:
F.jacobian(func, (r_vec, t_vec, mu_w_tensor))

(tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[ 0.2000,  0.0000, -0.1200,  0.0800],
         [ 0.0000,  0.2000, -0.1600,  0.0400],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]))

In [22]:
dmuI_dTcw: 
 [[ 0.2   0.   -0.12 -0.48  1.36 -0.8 ]
 [ 0.    0.2  -0.12 -1.48  0.36  0.6 ]]

SyntaxError: invalid syntax (3213940207.py, line 1)

In [32]:
F.jacobian(func_2, (vec, mu_w_tensor))

tensor([3., 4., 5., 1.], grad_fn=<MvBackward0>)
tensor([0.6000, 0.8000, 1.0000], grad_fn=<DivBackward0>)


(tensor([[ 0.2000,  0.0000, -0.1200,  0.0800,  1.2000, -1.2400],
         [ 0.0000,  0.2000, -0.1600, -1.4600,  1.1000, -0.2200],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]),
 tensor([[ 0.2000,  0.0000, -0.1200,  0.0800],
         [ 0.0000,  0.2000, -0.1600,  0.0400],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]))

In [ ]:
F.jacobian(